In [ ]:
include("/Users/meesvandartel/Desktop/LPR FinGrow/Reprod Paper/GrowthBasic.jl")
using .GrowthBasic
using Agents
using Plots
using HypothesisTests
using ProgressMeter
using Statistics
using Base.Threads
using StatsBase

In [ ]:
#ADF tests on aggregate and differences series respectively
MC = 1
dates = 1:500
agg_p = []
growth_p=[]
growth_vectors=[]

@showprogress for iter in 1:MC
    model = GrowthBasic.initialize_econ(;seed = rand(Int))
    GDP_series = []
    for t in dates
        GDP = 0
        for agent in allagents(model)
            Q = agent.Q
            GDP += Q
        end
        push!(GDP_series, log(GDP))
        step!(model)
    end
    GDP_series = convert(Vector{Float64}, GDP_series)
    GDP_growth = diff(GDP_series)
    push!(growth_vectors, GDP_growth)

    test_agg = ADFTest(GDP_series, :constant, :0)
    test_growth = ADFTest(GDP_growth,:constant,:0)
    push!(agg_p, pvalue(test_agg))
    push!(growth_p, pvalue(test_growth))
end


In [ ]:
c_agg = count(x -> x>0.05, agg_p)
c_growth = count(x -> x>0.05, growth_p)

println("Share of accepted H0= unit root for aggregate: $((c_agg/MC)*100)%")

println("Share of accepted H0= unit root for differenced: $((c_growth/MC)*100)&")

#Aggregare is non-stationary, differences is stationary just as in paper

In [ ]:
MC = 100
dates = 500
growth_vectors=Vector{Vector{Float64}}(undef, MC)

@showprogress for iter in 1:MC
    model = GrowthBasic.initialize_econ(;seed = rand(Int))
    GDP_series = Vector{Float64}(undef, dates)
    for t in 1:dates
        GDP = 0.0
        @inbounds for agent in allagents(model)
            GDP += agent.Q
        end
        GDP_series[t] = log(GDP)
        step!(model)
    end
    GDP_growth = diff(GDP_series)
    growth_vectors[iter] = GDP_growth

end

print(growth_vectors)

In [ ]:
test = mean(growth_vectors)
print(test)

In [ ]:
acf_values = autocor(test, collect(1:30); demean = true)
print(acf_values)

plot(acf_values, title="Autocorrelation Function", xlabel="Lag", ylabel="ACF Value")